In [2]:
def get_defining_set(n, pol, field):
    #defining_set = [field(0)]
    defining_set = []
    
    while len(defining_set) != n:
        aux = field.random_element()
        if pol(aux) != 0 and aux not in defining_set:
            defining_set = defining_set + [aux]
    
    return defining_set

In [3]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    r"""
    Implementation of Goppa codes.
    
    INPUT:
    - ``field`` -- finite field on which `self` is defined.
    - ``generating_pol`` -- a monic polynomial with coefficients in
    a finite field `\GF{p^m}` extending from `field`.
    
    - ``defining_set`` -- tuple of n distinct elements of `\GF{p^m}`
    that are not roots of `generating_pol`
    """
    def __init__(self, defining_set, generating_pol, field):
        """
        Initialize.
        """
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = field
        self._field_L = generating_pol.base_ring()
        
        if (not self._field.is_field() or not self._field.is_finite()):
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
    
    def _repr_(self):
        """
        Representation of a Goppa code.
        """
        return "[{}, {}] Goppa code".format(self._length, self.get_dimension())
    
    def get_generating_pol(self):
        """
        Return the generating polynomial of ``self``.
        """ 
        return self._generating_pol
    
    def get_defining_set(self):
        """
        Return the defining set of ``self``.
        """ 
        return self._defining_set
    
    def get_parity_pol(self):
        """
        Return the parity polynomial of ``self``.
        """
        parity_pol = list()
        
        for elem in self._defining_set:
            parity_pol.append((self._generating_pol.parent().gen() - elem).inverse_mod(self._generating_pol))
        
        return parity_pol
    
    def get_parity_check_matrix(self):
        """
        Return a parity check matrix of ``self``.
        """
        V, from_V, to_V = self._field_L.vector_space(self._field, map = True)

        parity = self.get_parity_pol()

        vector_L = []
        for i in range(len(parity)):
            vector_L = vector_L + parity[i].list()

        vector_F = []
        for i in range(len(vector_L)):
            vector_F = vector_F + to_V(vector_L[i]).list()

        matriz_F = matrix(self._length, vector_F)
        matriz_F = matriz_F.T
        
        return matriz_F
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of ``self``.
        """
        H = self.get_parity_check_matrix()
        G = transpose(H).left_kernel().basis_matrix()
        
        return G
    
    def get_dimension(self):
        """
        Return the dimension of the code.
        """
        
        return rank(self.get_generator_matrix())

class GoppaEncoder(Encoder):
    r"""
    Encoder for Goppa codes
    
    INPUT:
    - ``code`` -- code associated with the encoder
    """
    def __init__(self, code):
        """
        Initialize.
        """
        super(GoppaEncoder, self).__init__(code)
        
    def _repr_(self):
        """
        Representation of a encoder for a Goppa code
        """
        return "Encoder for {}".format(self.code())
    
    def get_generator_matrix(self):
        """
        Return a generador matrix of the code
        """
        return self.code().get_generator_matrix()
    
    def encode (self, m):
        """
        Return a codeword
        
        INPUT:
        - ``m``: a vector to encode
        """
        return m * self.get_generator_matrix()
    
Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder

class GoppaDecoder(Decoder):
    r"""
    Decoder for Goppa codes
    
    INPUT:
    - ``code``: code associated with the decoder
    """
    
    def __init__(self, code):
        """
        Initialize
        """
        super(GoppaDecoder, self).__init__(code, code.ambient_space(), "GoppaDecoder")
                
        self._generating_pol = self.code().get_generating_pol()
        self._defining_set = self.code().get_defining_set()
    
    def _repr_(self):
        """
        Representation of a decoder for a Goppa code
        """
        return "Decoder for {}".format(self.code())
    
    def get_syndrome(self, c):
        """
        Return the syndrome polynomial
        
        INPUT:
        - ``c``: a element of the input space of ``self``.
        """
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        
        h = self.code().get_parity_pol()
        
        syndrome = 0
        
        for i in range(len(h)):    
            syndrome = syndrome + embFL(c[i])*h[i]
            
        return syndrome
    
    def get_generating_pol(self):
        """
        Return the generating polynomial
        """
        return self._generating_pol
    
    def decode_to_code(self, word):
        r"""
        Corrects the errors in ``word`` and returns a codeword.
        INPUT:
        - ``word`` -- a codeword of ``self``
        """
        i = 1
        field = self.code()._field
        field_L = self.code()._field_L
        
        embFL = FiniteFieldHomomorphism_generic(Hom(field,field_L))
        secLF = embFL.section()

        # Step 1
        S = self.get_syndrome(word)
        
        if S == 0:
            return word

        # Step 2
        r_prev = self.get_generating_pol()
        t = floor(self.get_generating_pol().degree()/2)
        r_i = S
        U_prev = 0
        U_i = 1
        
        # Steps 3 and 4
        while r_i.degree() >= t:
            (q, r) = r_prev.quo_rem(r_i)
            aux_r_i = r_i
            aux_U_i = U_i
            r_i = r
            U_i = q * U_i + U_prev
            r_prev = aux_r_i
            U_prev = aux_U_i  
            i += 1
            
        # Step 5
        # make sigma monic     
        sigma = U_i/U_i.coefficients()[-1]
        eta = (-1)^i * r_i / U_i.coefficients()[-1]
        
        # roots of sigma are the locations of the errors
        roots_loc = []
        
        for root in sigma.roots():
            roots_loc = roots_loc + [self._defining_set.index(root[0])]
        
        error = [0] * len(self._defining_set)
        x = self.get_generating_pol().parent().gen()
        sigma_diff = sigma.diff(x)
        
        for i in range(0, len(sigma.roots())):
            error[roots_loc[i]] = secLF(eta.subs(x=sigma.roots()[i][0])/(sigma_diff.subs(x=sigma.roots()[i][0])))
        
        x = word - vector(self.code()._field, error)
        
        return x

Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

# Calcular una matriz generadora

In [55]:
F = GF(2)
L = GF(2^8)
a = L.gen()
b = F.gen()
R.<x> = L[]
g = x^5 + a*x^2 + 1
n = 50
defining_set = get_defining_set(n, g, L)

# Goppa code
C = Goppa(defining_set, g, F)

print("Goppa code:")
print(C)

G = C.get_generator_matrix()
show(G)

Goppa code:
[50, 10] Goppa code


10 x 50 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

# GGA

In [4]:
"""F = GF(2^2)
a = F.gen()

G = matrix(F, [
    [a,1,a+1,0,0,a,1,1,a+1,a],
    [a,0,a,1,1,a+1,a,a+1,a+1,a],
    [0,1,a+1,1,a+1,a,0,a,a,a+1],
    [a,1,1,0,0,1,a+1,a,a+1,1]
])"""

'F = GF(2^2)\na = F.gen()\n\nG = matrix(F, [\n    [a,1,a+1,0,0,a,1,1,a+1,a],\n    [a,0,a,1,1,a+1,a,a+1,a+1,a],\n    [0,1,a+1,1,a+1,a,0,a,a,a+1],\n    [a,1,1,0,0,1,a+1,a,a+1,1]\n])'

In [9]:
def fitness_vector(v):
    f = 0
    
    for i in v:
        if i != 0:
            f += 1
    
    return f

In [10]:
def fitness_matrix(M):
    for i in M:
        f_min = M.ncols()
        f = fitness_vector(i)
        
        if f < f_min:
            f_min = f
    
    return f_min

In [11]:
def fitness_permutation(G, c):
    n = G.nrows()
    m = G.ncols()
    Gx = matrix(F, n, m)
    for i in range(0, m):
        col = c[i] - 1

        for j in range(0, n):
            Gx[j,col] = G[j,i]

    return fitness_matrix(Gx.rref())

In [12]:
def random_sol():
    tam = G.ncols()
    sol = []
    
    while len(sol) != tam:
        r = randint(1, tam)
        
        if r not in sol:
            sol = sol + [r]
            
    return sol

In [13]:
def crossover(p1, p2):
    return Permutation(p1) * Permutation(p2), Permutation(p2) * Permutation(p1)

In [14]:
def mutation(p):
    n = len(p)
    k = randint(0, n - 1)
    k1 = randint(0, k - 1)
    k2 = randint(k, n - 1)

    return Permutation(k1, k2) * p

In [15]:
def valid(c):
    return c != 0

In [21]:
def GGA(N, pc, MaxReinit):
    t = 0
    #Maxt = 500000
    Maxt = 1000
    Pt = []
    reinit = 0
    
    # Initialize the Population P(t)
    for i in range(0, N):
        sol = random_sol()
        
        while not valid(sol):
            sol = random_sol()
            
        Pt = Pt + [sol]
    
    Pt = Matrix(Pt)
    
    # Evaluate
    fitnessPt = []
    for Pti in Pt:
        fitnessPt = fitnessPt + [fitness_permutation(G, Pti)]
        
    while(t < Maxt):  
        Ptsig = []
        fitnessPtsig = []
        
        # Binary tournament selection
        parents = []
        for i in range(0, N):            
            ind1 = Pt[randint(0, N-1)] # TODO: es uniforme?
            ind2 = Pt[randint(0, N-1)] # TODO: es uniforme?
            
            if (fitness_permutation(G, ind1) >= fitness_permutation(G, ind2)):
                parents = parents + [ind2]
            else:
                parents = parents + [ind1]
        
        parents = Matrix(parents)
        
        # Generation
        for i in range(0, N/2):
            nCrossovers = pc * N/2 # number of crossovers
            
            if nCrossovers > 0: # TODO: es uniforme?
                c1, c2 = crossover(parents[2*i], parents[2*i + 1])
                nCrossovers -= 1
            else:
                c1 = mutation(copy(parents[2*i]))
                c2 = mutation(copy(parents[2*i + 1]))
            
            # Valid
            while not valid(c1):
                c1 = random_sol()
            while not valid(c2):
                c2 = random_sol()
            
            # Update
            Ptsig = Ptsig + [c1]
            Ptsig = Ptsig + [c2]
        
        Ptsig = Matrix(Ptsig)
        
        # Evaluate
        for Ptsigi in Ptsig:
            fitnessPtsig = fitnessPtsig + [fitness_permutation(G, Ptsigi)]
        
        # No improvement
        if min(fitnessPtsig) > min(fitnessPt):
            worst = fitnessPtsig.index(max(fitnessPtsig))
            best = fitnessPt.index(min(fitnessPt))
            Ptsig[worst] = Pt[best]
            fitnessPtsig[worst] = fitnessPt[best]
            reinit += 1
        else:
            reinit = 0
        
        # Restart
        if reinit >= MaxReinit:
            Ptsig = []
            for i in range(0, N-1):
                Ptsig = Ptsig + [random_sol()]
            
            best = fitnessPt.index(min(fitnessPt))
            Ptsig = Ptsig + [Pt[best]]
            Ptsig = Matrix(Ptsig)
            
            # Evaluate
            fitnessPtsig = []
            for Ptsigi in Ptsig:
                fitnessPtsig = fitnessPtsig + [fitness_permutation(G, Ptsigi)]
        
        fitnessPt = fitnessPtsig
        Pt = Ptsig
        t += 1
    
    best = fitnessPt.index(min(fitnessPt))
    
    return Pt[best], fitness_permutation(G, Pt[best])

In [77]:
#GGA(400, 0.7, 100000)
GGA(40, 0.7, 1000)

((23, 10, 17, 24, 33, 2, 20, 50, 46, 36, 29, 27, 5, 22, 25, 8, 47, 40, 35, 39, 21, 3, 41, 28, 38, 7, 34, 18, 44, 12, 19, 6, 49, 26, 4, 13, 31, 9, 11, 42, 45, 48, 32, 37, 14, 1, 30, 15, 43, 16),
 11)

# CHC

In [31]:
def distance(x, y):
    assert(len(x) == len(y))
    
    n = len(x)
    count = 0
    for i in range(0, n):
        if x[i] != y[i]:
            count += 1
            
    return count

In [46]:
def distanceP(P, tau):
    d = 0
    Maxdist = 0
    count = 0
    for i in range(0, P.nrows()):
        for j in range(i, P.nrows()):
            dist = distance(P[i], P[j])
            
            if dist > Maxdist:
                Maxdist = dist
            
            d += dist
            count += 1
    
    d = d * 1.0/count
    dec = tau * Maxdist
    
    return d, dec

In [95]:
def CHC(N, tau):
    t = 0
    #Maxt = 500000
    Maxt = 5000
    Pt = []
    reinit = 0
    
    # Initialize the Population P(t)
    for i in range(0, N):
        sol = random_sol()
        
        while not valid(sol):
            sol = random_sol()
            
        Pt = Pt + [sol]
    
    Pt = Matrix(Pt)
    
    # Evaluate
    fitnessPt = []   
    for Pti in Pt:
        fitnessPt = fitnessPt + [fitness_permutation(G, Pti)]
    
    # Distance
    d, dec = distanceP(Pt, tau)
        
    while(t < Maxt):
        Ct = []
        Ptsig = []
        fitnessCt = []
        fitnessPtsig = []
        
        # Parent selection
        parents = []
        for i in range(0, N):            
            p = randint(0, N-1)
            parents = parents + [Pt[p]]
        
        parents = Matrix(parents)
        
        # Generation
        for i in range(0, N/2):            
            if distance(parents[2*i], parents[2*i + 1]) < d:
                c1, c2 = crossover(parents[2*i], parents[2*i + 1])
                Ct = Ct + [c1]
                Ct = Ct + [c2]
        
        Ct = Matrix(Ct)
        
        # Evaluate
        for Cti in Ct:
            fitnessCt = fitnessCt + [fitness_permutation(G, Cti)]
        
        if Ct.nrows() == 0:
            Ptsig = Pt
            fitnessPtsig = fitnessPt
        else:
            fitnessPtaux = copy(fitnessPt)
            fitnessCtaux = copy(fitnessCt)
            
            for i in range(0, N):
                bestPt = min(fitnessPtaux)
                bestCt = min(fitnessCtaux)

                if bestPt < bestCt:
                    index = fitnessPtaux.index(bestPt)
                    Ptsig = Ptsig + [Pt[index]]
                    fitnessPtsig = fitnessPtsig + [bestPt]
                    fitnessPtaux[index] = Infinity
                else:
                    index = fitnessCtaux.index(bestCt)
                    Ptsig = Ptsig + [Ct[index]]
                    fitnessPtsig = fitnessPtsig + [bestCt]
                    fitnessCtaux[index] = Infinity
        
            Ptsig = Matrix(Ptsig)
        
        # No improvement
        if Pt == Ptsig:
            d = d - dec
            
            if d <= 0:
                Ptsig = []
                for i in range(0, N-1):
                    Ptsig = Ptsig + [random_sol()]

                best = fitnessPt.index(min(fitnessPt))
                Ptsig = Ptsig + [Pt[best]]
                Ptsig = Matrix(Ptsig)
                
                # Distance
                d, dec = distanceP(Pt, tau)
        
        fitnessPt = fitnessPtsig
        Pt = Ptsig
        t += 1
    
    best = fitnessPt.index(min(fitnessPt))
    
    return Pt[best], fitness_permutation(G, Pt[best])

In [96]:
#CHC(400, 0.7, 100000)
CHC(50, 0.7)

((50, 13, 44, 18, 19, 28, 30, 1, 45, 43, 29, 15, 32, 33, 48, 2, 36, 11, 9, 34, 47, 12, 40, 24, 17, 5, 20, 14, 37, 21, 7, 10, 16, 6, 39, 42, 3, 46, 41, 31, 27, 38, 25, 23, 35, 22, 49, 26, 8, 4),
 19)